In [36]:
import io
import os
import sys
import requests
from collections import OrderedDict 
from collections import defaultdict
import math
import random
import numpy as np

In [37]:
text = "natural language processing and machine learning is fun and exciting"
corpus = [word.lower() for word in text.split()]
corpus

['natural',
 'language',
 'processing',
 'and',
 'machine',
 'learning',
 'is',
 'fun',
 'and',
 'exciting']

In [38]:
settings = {
    'window_size':2,
    'n':10, #单词嵌入的维度
    'epochs': 50,
    'learning_rate':0.01}

In [39]:
class word2vec():
    def __init__(self):
        self.n = settings['n']
        self.lr = settings['learning_rate']
        self.epochs = settings['epochs']
        self.window = settings['window_size']
    def generate_training_data(self,settings,corpus):
        word_counts = defaultdict(int)
        for row in corpus:
            for word in row:
                word_counts[word] +=1
        self.v_count = len(word_counts.keys())
        self.words_list = list(word_counts.keys())
        self.word_index = dict((word,i)for i ,word in enumerate(self.words_list))
        self.index_word = dict((i,word)for i ,word in enumerate(self.words_list))

        trainging_data = []
        for sentence in corpus:
            sent_len= len(sentence)
            for i ,word in enumerate(sentence):
                w_target = self.word2onehot(sentence[i])
                word_context = []
                for j in range (i-self.window,i+self.window+1):
                    if j != i and j <= sent_len-1 and j > 0:
                        word_context.append(self.word2onehot(sentence[j]))
                    trainging_data.append([w_target,word_context])
        return np.array(trainging_data)

    def word2onehot(self,word):
        word_vec = [0 for i in range(0,self.v_count)]
        word_index = self.word_index[word]
        word_vec[word_index]=1 
        return word_vec
    
    def forward_pass(self,x):
        h = np.dot(self.w1.T,x)
        u = np.dot(self.w2.t,h)
        y_c = self.softmax(u)
        return y_c , h ,u
        
    def softmax(self,x):
        e_x = np.exp(x - np.max(x))
        return e_x / e_x.sum(axis = 0)


    def backprop(self, e, h, x):
        dl_dw2 = np.outer(h, e)  
        dl_dw1 = np.outer(x, np.dot(self.w2, e.T))
        self.w1 = self.w1 - (self.eta * dl_dw1)
        self.w2 = self.w2 - (self.eta * dl_dw2)


    def trianing(self,training_data):
        self.w1=np.random.uniform(-1,1,(self.v_count,self.n))
        self.w2=np.random.uniform(-1,1,(self.n,self.v_count))
        for i in range(self.epochs):
            self.loss = 0
            for w_t , w_c in trainging_data:
                y_pred , h ,u =self.forward_pass(w_t)
                EI = np.sum([np.subtract(y_pred,word)for word in w_c],axixs=0)
                self.backprop(EI,h,w_t)
                self.loss += -np.sum([u[word.index(1)] for word in w_c]) + len(w_c) * np.log(np.sum(np.exp(u)))
        print('Epoch:',i,"loss:",self.loss)

    # input a word, returns a vector (if available)
    def word_vec(self, word):
        w_index = self.word_index[word]
        v_w = self.w1[w_index]
        return v_w


    # input a vector, returns nearest word(s)
    def vec_sim(self, vec, top_n):

        # CYCLE THROUGH VOCAB
        word_sim = {}
        for i in range(self.v_count):
            v_w2 = self.w1[i]
            theta_num = np.dot(vec, v_w2)
            theta_den = np.linalg.norm(vec) * np.linalg.norm(v_w2)
            theta = theta_num / theta_den

            word = self.index_word[i]
            word_sim[word] = theta

        words_sorted = sorted(word_sim.items(), key=lambda kv :kv[1] ,reverse=True)

        for word, sim in words_sorted[:top_n]:
            print (word, sim)
            
        pass

    # input word, returns top [n] most similar words
    def word_sim(self, word, top_n):
        
        w1_index = self.word_index[word]
        v_w1 = self.w1[w1_index]

        # CYCLE THROUGH VOCAB
        word_sim = {}
        for i in range(self.v_count):
            v_w2 = self.w1[i]
            theta_num = np.dot(v_w1, v_w2)
            theta_den = np.linalg.norm(v_w1) * np.linalg.norm(v_w2)
            theta = theta_num / theta_den

            word = self.index_word[i]
            word_sim[word] = theta

        words_sorted = sorted(word_sim.items(), key=lambda kv :kv[1] , reverse=True)

        for word, sim in words_sorted[:top_n]:
            print (word, sim)
            
        pass


In [40]:
w2v = word2vec()
trainging_data = w2v.generate_training_data(settings,corpus)
# vec = w2v.word_vec("machine")

<ipython-input-39-af995f710362>:27: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return np.array(trainging_data).astype("float64")


ValueError: setting an array element with a sequence.